In [24]:
from binance.um_futures import UMFutures
import plotly.graph_objects as go
import requests
import json
import numpy as np
import pandas as pd
from pandas import Timestamp
from sklearn.linear_model import LinearRegression
import datetime
from chart import Chart
um_futures_client = UMFutures()
um_futures_client = UMFutures("iPQMe46exUy10KiBaBahQT7ow1uzb9jaxlKj19Bg5BI8JEwJL5bw9LCvJtfKuVbP","J4FGJQdGPpNGm8jpnj3IuFuDJuAe7FIa3cM51L4z662UYux3qD0ByLJ0bfIltZvK")

Source = https://github.com/binance/binance-futures-connector-python/tree/main (UMFutures)

Documentation = https://binance-docs.github.io/apidocs/futures/en/#public-endpoints-info (UMFutures)

In [25]:
def convert_to_df(data):
    data_df=pd.DataFrame(np.asarray(data,dtype=np.float64),columns=["Open Time","Open","High","Low","Close","Volume","Close Time","Quote Asset Volume","Number of Trades","Taker Buy Volume","Taker Buy Quote Asset Volume","Ignore"])
    data_df['Open Time'] = pd.to_datetime(data_df['Open Time'], unit='ms')
    data_df['Close Time'] = pd.to_datetime(data_df["Close Time"],unit='ms').dt.ceil("min")
    return data_df.drop("Ignore",axis=1)

In [26]:
data=convert_to_df(um_futures_client.continuous_klines("CHZUSDT",contractType="PERPETUAL", interval="15m"))
chz_chart = Chart(data)

TypeError: list indices must be integers or slices, not str

In [ ]:
chz_chart.plot()

ValueError: 
    Invalid value of type 'builtins.type' received for the 'type' property of layout.xaxis
        Received value: <class 'type'>

    The 'type' property is an enumeration that may be specified as:
      - One of the following enumeration values:
            ['-', 'linear', 'log', 'date', 'category',
            'multicategory']

In [ ]:
line = chz_chart.gradient_descent(data[200:499])

ValueError: Expected 2D array, got 1D array instead:
array=['2024-02-15T07:30:00.000000000' '2024-02-15T07:45:00.000000000'
 '2024-02-15T08:00:00.000000000' '2024-02-15T08:15:00.000000000'
 '2024-02-15T08:30:00.000000000' '2024-02-15T08:45:00.000000000'
 '2024-02-15T09:00:00.000000000' '2024-02-15T09:15:00.000000000'
 '2024-02-15T09:30:00.000000000' '2024-02-15T09:45:00.000000000'
 '2024-02-15T10:00:00.000000000' '2024-02-15T10:15:00.000000000'
 '2024-02-15T10:30:00.000000000' '2024-02-15T10:45:00.000000000'
 '2024-02-15T11:00:00.000000000' '2024-02-15T11:15:00.000000000'
 '2024-02-15T11:30:00.000000000' '2024-02-15T11:45:00.000000000'
 '2024-02-15T12:00:00.000000000' '2024-02-15T12:15:00.000000000'
 '2024-02-15T12:30:00.000000000' '2024-02-15T12:45:00.000000000'
 '2024-02-15T13:00:00.000000000' '2024-02-15T13:15:00.000000000'
 '2024-02-15T13:30:00.000000000' '2024-02-15T13:45:00.000000000'
 '2024-02-15T14:00:00.000000000' '2024-02-15T14:15:00.000000000'
 '2024-02-15T14:30:00.000000000

In [ ]:
chz_chart.add_line("t1",data.iloc[200]["Open Time"],data.iloc[499]["Close Time"],data.iloc[200]["Low"]*line[0] + line[1], data.iloc[499]["Low"]*line[0]+line[1])

In [ ]:
type(data.iloc[0]["Close Time"])

pandas._libs.tslibs.timestamps.Timestamp

In [ ]:
print(min_dif)

-0.002914721481188959


In [ ]:
print(data.iloc[200]["Low"]*line[0] + line[1])

0.10046741004789585


In [ ]:
chz_chart.plot()

In [ ]:
chz_chart.delete_line("t1")

In [ ]:
chz_chart.auto_draw_line(data[200:499])

TypeError: can only concatenate str (not "int") to str